In [ ]:
autosave 0

# Linear inversion examples

## Preliminary steps

### Importing necessary libraries

In [ ]:
#Adding library modules to PYTHONPATH
import sys
sys.path.append("../python")
import numpy as np
#Inversion library-related modules
import pyVector as Vec
import pyOperator as Op
from pyNpOperator import MatrixOp
from pyLinearSolver import LCGsolver as LCG
from pyLinearSolver import SymLCGsolver
import pyProblem as Prblm
from pyStopper import BasicStopper as Stopper
#Plotting library
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline
params = {
    'image.interpolation': 'nearest',
    'image.cmap': 'gray',
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'axes.labelsize': 12, # fontsize for x and y labels (was 10)
    'axes.titlesize': 12,
    'font.size': 12, # was 10
    'legend.fontsize': 12, # was 10
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
}
matplotlib.rcParams.update(params)

## Instantiation of vectors and operator

For testing the library we will be using a discretized version of the following operator:
\begin{align}
y = \frac{d^2f(x)}{dx^2},
\end{align}
in which we simply compute the second-order derivative of a function $f(x)$. 

In [ ]:
N=200 #Number of points of the function f(x)
dx=1.0 #Sampling of the function
D2 = np.matrix(np.zeros((N,N),dtype=np.float64)) #Matrix containing the discretization of the derivative operator
#The stencil used is simply: (f(ix-1)-2f(ix)+f(ix+1))/(dx*dx)
np.fill_diagonal(D2, -2/(dx*dx))
np.fill_diagonal(D2[1:], 1/(dx*dx))
np.fill_diagonal(D2[:,1:], 1/(dx*dx))
f = Vec.vectorIC(np.zeros((N,1),dtype=np.float64)) #Initializing numpy-based vector for f(x)
y = Vec.vectorIC(np.zeros((N,1),dtype=np.float64)) #Initializing numpy-based vector for y
D2Op = MatrixOp(D2,f,y)

Before we set any inversion problem, we study some of the properties of the constructed operator Deriv2Op.

In [ ]:
#Verifying operator adjointness through dot-product test
D2Op.dotTest(verbose=True)
#Computing maximum and minimum eigenvalues of the operator using the power iteration method and 
#compare them against the ones computed using numpy
egsOp=D2Op.powerMethod(verbose=False,eval_min=True,tol=1e-300)
egsNp,_=np.linalg.eig(D2)
egsNp = egsNp[egsNp.argsort()[::-1]] #Sorting the eigenvalues
print("\nMaximum eigenvalue: %s (Power method), %s (NUMPY)"%(egsOp[0],egsNp[-1]))
print("Minimum eigenvalue: %s (Power method), %s (NUMPY)"%(egsOp[1],egsNp[0]))

We can see that the matrix is negative definite. The small mismatch in the estimated eigenvalues is due to the dependence of the power method on the initial random eigenvector.

## Inversion tests

We will now focus our attention on inverting a function knowning its second-order derivative. In this case we will assume that $y$ is constant and equal to $1$. Therefore, we expect to obtain a parabola with positive curvature. Given the chosen boundary conditions we know that the matrix is invertible since all eigenvalues have the same sign and are different then zero.
We will solve the following objective functions using linear conjugate-gradient methods:
\begin{equation*}
\phi_1(\mathbf{f}) = \frac{1}{2}\|D_2\mathbf{f}-\mathbf{y}\|_2^2
\end{equation*}
and
\begin{equation*}
\phi_2(\mathbf{f}) = \frac{1}{2}\mathbf{f}^T D_2 \mathbf{f} - \mathbf{f}^{T} \mathbf{y},
\end{equation*}
where $D_2$ represents the discretized second-order derivative operator, while $\mathbf{f}$ and $\mathbf{y}$ are the discretized representations of $f$ and $y$, respectively.

In [ ]:
y.set(1.0) # y = 1
#Note that f = 0
Phi1 = Prblm.ProblemL2Linear(f.clone(),y,D2Op)
Phi2 = Prblm.ProblemLinearSymmetric(f.clone(),y,D2Op)

### Instantiation of solver objects

First, we create two different solver object for solving the two inversion problem stated above.

In [ ]:
#Create stopping criteria and related object
niter = 2000
Stop  = Stopper(niter=niter)
#Create LCG solver
LCGsolver = LCG(Stop)
LCGsolver.setDefaults(save_obj=True) #Saving objective function within the solver
#Create LCG solver for symmetric systems
SLCG = SymLCGsolver(Stop)
SLCG.setDefaults(save_obj=True)

Secondly, we run the solvers to minimize the objective functions previously defined.

In [ ]:
LCGsolver.run(Phi1,verbose=True)

In [ ]:
SLCG.run(Phi2,verbose=True)

Finally, we can look at the results.

In [ ]:
%matplotlib inline
plt.plot(np.log10(LCGsolver.obj/LCGsolver.obj[0]))
plt.title("LCG convergence")
plt.xlabel("Iteration #",fontsize=14)
plt.ylabel("$log_{10}$($\phi_i/\phi_0$)",fontsize=14)
ax = plt.gca() 
ax.autoscale(enable=True, axis='x', tight=True)

In [ ]:
%matplotlib inline
plt.plot(SLCG.obj)
plt.title("Symmetric LCG convergence")
plt.xlabel("Iteration #",fontsize=14)
plt.ylabel("$\phi_i$)",fontsize=14)
ax = plt.gca() 
ax.autoscale(enable=True, axis='x', tight=True)

Also, let's compare the two inverted functions with the analytical solution.
To find the solution for the continuous case we need three conditions:
\begin{equation}
\frac{d^2f(x)}{dx^2}=1,\\
f(x=0)=0,\\
f(x=x_f)=0.
\end{equation}
$x = 0$ and $x = x_f$ are not sampled and lay outside of the interval $\mathbf{x}$.

In [ ]:
X = np.linspace(dx,N*dx,N)
alpha = 0.5
beta  = -(X[-1]+dx)*0.5
gamma = 0.0
f_an  = alpha * X * X + beta * X + gamma

In [ ]:
_ = plt.plot(X,Phi1.model.getNdArray(),linewidth=3,label="$f(x)$ from $\phi_1$")
_ = plt.plot(X,Phi2.model.getNdArray(),linewidth=2,dashes=[6, 2],label="$f(x)$ from $\phi_2$")
_ = plt.plot(X,f_an,'k',linewidth=2,dashes=[2, 2],label="Analytical solution")
ax = plt.gca() 
ax.autoscale(enable=True, axis='x', tight=True)
ax.legend()
plt.xlabel("x",fontsize=14)
plt.ylabel("$f(x)$",fontsize=14)
_ = plt.title("Inverted functions")

Now, let's try to solve both inversions using the inverse of $D_2$ as a preconditioner

In [ ]:
PrecOp = MatrixOp(np.linalg.inv(D2),f,y) # P = [D_2]^-1
Phi1Prec = Prblm.ProblemL2Linear(f.clone(),y,D2Op,prec=Op.ChainOperator(PrecOp,PrecOp)) #In this line we chain [D_2]^-1 twice
Phi2Prec = Prblm.ProblemLinearSymmetric(f.clone(),y,D2Op,prec=PrecOp)

In [ ]:
LCGsolver.setDefaults() # Re-setting default solver values
SLCG.setDefaults() # Re-setting default solver values

In [ ]:
LCGsolver.run(Phi1Prec,verbose=True)

In [ ]:
SLCG.run(Phi2Prec,verbose=True)

As expected, we converge to the global minimum in effectively one iteration.